# Chacha

In [8]:
from typing import List, Tuple
from scripts.utils import *
from scripts.math import sum32

## Pad function

In [9]:
_c = [0x61707865, 0x3320646e, 0x79622d32, 0x6b206574]

def pad(s: int, j: int, n: int) -> int:
    """A padding function that combies a 256 bits seed s with a 64 bit counter j (j0, j1 of 32 bits) and a b4 bit nonce (n0, n1 of 32 bits) to form a 512 bit block denoted x0, ..., x15 of 32 bits
    
    Args:
        s: Seed (256 bits, 8 words of 32 bits)
        j: counter (64 bits)
        n: Nonce (64 bits)

    Returns:
        512 bits (16 words 32 bits)
    """

    # Convert to words
    s = to_words(s, 32, 8)
    j = to_words(j, 32, 2)
    n = to_words(n, 32, 2)

    return [
        _c[0], _c[1], _c[2], _c[3],
        s[0], s[1], s[2], s[3],
        s[4], s[5], s[6], s[7],
        j[0], j[1], n[0], n[1]
    ]

## Public permutation

### Quarterround function

In [10]:
def rot(w: int, r: int) -> int: 
    """Rotate lest for 32 bits

    Args:
        w: word to rotate
        r: I dont remember
    """

    mask = 0xffffffff
    return ((w << r) & mask) | (w >> (32 - r)) 

In [11]:
def QR(a, b, c, d):
    a = sum32(a, b)
    d = d ^ a 
    d = rot(d, 16)

    c = sum32(c, d)
    b = b ^ c
    b = rot(b, 12)

    a = sum32(a, b)
    d = d ^ a
    d = rot(d, 8)

    c = sum32(c, d)
    b = b ^ c
    b = rot(b, 7)
    
    return a, b, c, d

### Round function 

In [12]:
def round(i):
    i[0], i[4], i[8], i[12] = QR(i[0], i[4], i[8], i[12])
    i[1], i[5], i[9], i[13] = QR(i[1], i[5], i[9], i[13])
    i[2], i[6], i[10], i[14] = QR(i[2], i[6], i[10], i[14])
    i[3], i[7], i[11], i[15] = QR(i[3], i[7], i[11], i[15])

    i[0], i[5], i[10], i[15] = QR(i[0], i[5], i[10], i[15])
    i[1], i[6], i[11], i[12] = QR(i[1], i[6], i[11], i[12])
    i[2], i[7], i[8], i[13] = QR(i[2], i[7], i[8], i[13])
    i[3], i[4], i[9], i[14] = QR(i[3], i[4], i[9], i[14])

    return i

### Perm function 

In [13]:

def perm(x: int, ROUNDS: int = 20): 
    """Permutation 

    Args:
        x: 512 bits (16 words 32 bits)
        ROUNDS: Number of rounds
            - Default 20 (ChaCha20/20)
            - 8 (ChaCha20/8)
            - 12 (ChaCha20/12)
            
    Returns:
        512 bits (16 words 32 bits)
    """
    
    i = x
    
    for _ in range(0, ROUNDS, 2): 
        round(i)
    return i


## Psudorandom generator

In [14]:
L = 1 # 
def G(s: int, n: int):
    r = [0]*L

    for j in range(L):
        h = pad(s, j, n) # 512 bits
        pi = perm(h) # 512 bits

        r[j] = [0]*16
        for i in range(16):
           r[j] = from_words(pi, 32) + from_words(h, 32)
           
    return r

## Example

In [15]:
k = G(0x112233445566778899aabbccddeeff00, 0x0123456789abcdef)[0]
m = text_to_int("Hello word!")
c = m ^ k
assert(c ^ k == m), "Error"

In [17]:
hex(k)

'0x13c9d4da605d3fe8d20464a72aacf9c0d5deb50ca4fbb44aa4618510f5c8f406d4a97fd14f9c3bac3241ad4bedfc9c159ad3c371f5b9a1bee2b80cc198c25af52'

## References
- Class notes 
- [ChaCha, a variant of Salsa20 - Daniel J. Bernstein](https://cr.yp.to/chacha/chacha-20080128.pdf)